In [2]:
import pandas as pd 

In [28]:
pd.options.display.max_columns = 60
df = pd.read_csv('./carnivorous_diet.csv', encoding='latin1')

/var/folders/b5/0_tkhzk50998yq1mgzytzf8c0000gn/T/ipykernel_71770/1694165870.py:2: DtypeWarning: Columns (41) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./carnivorous_diet.csv', encoding='latin1')


In [27]:
# Remove index duplicate (recordID) and columns made of 99% 'Unknown's
df.drop(columns=['recordID', 'lifeStageCarni'], inplace=True)


In [33]:
# Remove all columns whose N/A represent more than half the dataset
def is_majority_na(columnName):
    return df[columnName].isna().sum() > len(df) / 2

for i in df.columns:
    if (is_majority_na(i)):
        print(f'Dropping column {i}')
        df.drop(columns=[i], inplace=True)

df.isna().sum()


recordID                           0
orderCarni                         0
familyCarni                        0
scientificNameCarni                0
commonNameCarni                    0
sexCarni                           0
lifeStageCarni                     0
foodType                           2
orderPrey                      10283
familyPrey                     11996
genusPrey                      13627
commonNamePrey                   423
domesticOrAgricultural           238
taxonRankPrey                      0
percentage                         0
dataBasisFromSource              350
sampleSizeScatStomachTissue     2810
startYear                       1334
endYear                         1334
startMonth                      9541
endMonth                        9528
fixedBetweenMonths              9721
season                          2224
country                          307
studyAreaSize                  10404
decimalLatitude                  222
decimalLongitude                 514
g